In [8]:
%pip -q install scikit-learn pandas numpy matplotlib

Note: you may need to restart the kernel to use updated packages.


In [69]:
# Surpress warnings
import warnings
warnings.filterwarnings('ignore')

In [70]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the data
data = pd.read_csv("data/titanic.csv")

# Select features from PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = data[features]
y = data["Survived"]

X.tail()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
886,2,male,27.0,0,0,13.00,S
887,1,female,19.0,0,0,30.00,S
888,3,female,NaN,1,2,23.45,S
889,1,male,26.0,0,0,30.00,C
890,3,male,32.0,0,0,7.75,Q


In [71]:
# Handle missing values
X["Age"].fillna(X["Age"].median(), inplace=True)
X["Embarked"].fillna(X["Embarked"].mode()[0], inplace=True)

# Encode categorical variables
le = LabelEncoder()
X["Sex"] = le.fit_transform(X["Sex"])
X["Embarked"] = le.fit_transform(X["Embarked"])

# Convert to numpy arrays
X = X.values
y = y.values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X[0]

array([ 3.  ,  1.  , 22.  ,  1.  ,  0.  ,  7.25,  2.  ])

In [72]:
# Assuming the SimpleDecisionTree class is already defined
from SimpleDecisionTree import SimpleDecisionTree

tree = SimpleDecisionTree()
tree.fit(X_train, y_train)

In [73]:
y_pred = tree.predict(X_test)
y_pred[:5]

[0, 0, 0, 1, 1]

In [74]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.80

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.66      0.74        74

    accuracy                           0.80       179
   macro avg       0.81      0.78      0.79       179
weighted avg       0.81      0.80      0.80       179



In [75]:
def visualize_tree(node, depth=0):
    if "value" in node:
        print("  " * depth + f"Predict: {node['value']}")
    else:
        feature_names = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
        print(
            "  " * depth
            + f"Split on {feature_names[node['feature_index']]} < {node['threshold']:.2f}"
        )
        print("  " * (depth + 1) + "Left:")
        visualize_tree(node["left"], depth + 2)
        print("  " * (depth + 1) + "Right:")
        visualize_tree(node["right"], depth + 2)


visualize_tree(tree.tree)

Split on Sex < 1.00
  Left:
    Split on Pclass < 3.00
      Left:
        Split on Age < 3.00
          Left:
            Split on Pclass < 2.00
              Left:
                Predict: 0
              Right:
                Predict: 1
          Right:
            Split on Age < 28.00
              Left:
                Split on Age < 24.00
                  Left:
                    Predict: 1
                  Right:
                    Predict: 1
              Right:
                Split on Age < 50.00
                  Left:
                    Predict: 1
                  Right:
                    Predict: 1
      Right:
        Split on Fare < 23.45
          Left:
            Split on Embarked < 2.00
              Left:
                Split on Fare < 15.50
                  Left:
                    Predict: 1
                  Right:
                    Predict: 1
              Right:
                Split on Fare < 7.75
                  Left:
                    Predi

In [76]:
def calculate_feature_importance(tree):
    def count_feature_usage(node, counts):
        if "feature_index" in node:
            counts[node["feature_index"]] += 1
            count_feature_usage(node["left"], counts)
            count_feature_usage(node["right"], counts)

    feature_counts = [0] * tree.n_features
    count_feature_usage(tree.tree, feature_counts)
    total = sum(feature_counts)
    return [count / total for count in feature_counts]


feature_importance = calculate_feature_importance(tree)
for i, importance in enumerate(feature_importance):
    print(f"Feature {features[i]}: {importance:.4f}")

Feature Pclass: 0.1304
Feature Sex: 0.0435
Feature Age: 0.4783
Feature SibSp: 0.0435
Feature Parch: 0.0000
Feature Fare: 0.2609
Feature Embarked: 0.0435
